In [ ]:
from pathlib import Path
import json

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

# from mat73 import loadmat as loadmat_mat73
# from mat4py import loadmat as loadmat_mat4py

import h5py
from scipy.io import loadmat as loadmat_scipy

# Session analysis
Let's start with a session to look at the files

## Files and their description. A first take

Peter Petersen, one of the members of the Buzsaki lab, has shared with us the following documents that contain the structure of the cell explorer format:

* [new format](https://cellexplorer.org/data-structure/)
* [old format](https://github.com/buzsakilab/buzcode/wiki/Data-Formatting-Standards)

He referred to the old format as buzcode.


In [ ]:
session_path = Path("/home/heberto/buzaki/e13_16f1_210302/")
session_files_path_list = list(session_path.iterdir())
session_files_path_list

Now as Cody mention, we have seen some experimental data from this lab already and we are familiar with the names.

In a [previous conversion](https://github.com/catalystneuro/buzsaki-lab-to-nwb/blob/master/buzsaki_lab_to_nwb/yuta_visual_cortex/files_documentation.ipynb) I found out that the matlab files contain the following information:



* `SleepState.states` : This can be considered processed data involving up-down intervals. This can be include as process data.
* `chanMap` : This seems to be concerned with information of the channels in the electrode. For example we find both the x and y coordinates of each of the channels. The structure of the files here is (1, n_channels) where n_channels is 64 for this setup.
* `session` : Contains behavioral info and general information related to the session such as the experimenter, the species, the strain and timestamps for the creation of the session.

As you see, we have the following descriptions missing:
* `.ripples.events.mat`: 
* `Behavior` : 

Which is something that we would do below.

#### Spike sorting
The files related to spike sorting were the following in a previous conversion.

For the previous conversion those were the files related to **cell explorer format / interface**:
* `metric_cell_info`
* `mono_res_cellinfo`
* `spikes.cell_info`

They don't seem equivalent to the ones here. We need to confirm that they are equivalent to the following files in this conversion:
* `cell_metrics.cellinfo.mat`
* `cell_metrics.cellinfo`
* `spikes.cellinfo`

This is done below.

Apparently, mono_res means Monosynaptic connections.



## Exploring some files



### `SleepState.states`

Let's now see now which matlab file opener works best

In [ ]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.SleepState.states.mat"
# Open file_path with loadmat_scipy from scipy
mat_file = loadmat_scipy(file_path, simplify_cells=True)
# Iterate over the keys and print the type of the values
for key in mat_file.keys():
    print(key, type(mat_file[key]))

This is a recursive structure. Let's print the keys, types and shapes (if numpy array) for exploration

In [ ]:

def build_keys_and_types(dictionary):
    output_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, dict):
            output_dict[key] = build_keys_and_types(value)
        elif isinstance(value, np.ndarray):
            if value.size > 10:
                output_dict[key] = {
                    'type': str(type(value)),
                    'shape': str(value.shape)
                }
            else:
                # Print small arrays
                output_dict[key] = {
                    'type': str(type(value)),
                    'value': str(value)
                }
        elif isinstance(value, list):
            if len(value) > 10:
                output_dict[key] = {
                    'type': str(type(value)),
                    'length': len(value)
                }
            else:
                # Print small lists
                output_dict[key] = {
                    'type': str(type(value)),
                    'value': str(value)
                }
        else:
            output_dict[key] = {
                "type": str(type(value)),
                "value": str(value),
            }
    return output_dict

# Define your sleep_state_dict here

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

# Dump to a file in the same folder
json_directory = Path.cwd() / "_json_files"
json_directory.mkdir(exist_ok=True)
with open(json_directory / 'sleep_state_dict.json', 'w') as f:
    f.write(json_output)
    

If you have a matlab licence, you can also just explore the file there. There are two things to look for, large arrays that might
correspond to behavorial data and metadata from the experiment.

Because I know the data from this lab I will be looking for the REM state.



In [ ]:
sleep_state_dict = mat_file["SleepState"]
json_dict = json.dumps(build_keys_and_types(sleep_state_dict), indent=4)

In [ ]:
sleep_state_dict["ints"]


In [ ]:
wake_state = sleep_state_dict["ints"]["WAKEstate"]
wake_state  # This is the start time and the end time of the wake state. Probably in frames.

So, we need to confirm the units and this can go as a `TimeIntervals` in a processing module

### `Behavior`

In [ ]:
file_path = file_path.parent / "e13_16f1_210302.Behavior.mat" 
file_path.is_file()

In [ ]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'behavior_dict.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

Importantly this contains a date in `date` and subject information in `animal`.
Surprisingly, this does not seem to contain any large vector as I expected

### `ripples.events.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.ripples.events.mat"
file_path.is_file()

In [ ]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'riples.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

This is a large file. We need to look into the paper to see what should be stored from here

### `chanMap.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "chanMap.mat'"
file_path.is_file()


In [ ]:
# Write this to a file for visualization
with open(json_directory / 'chanMap.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

There are some time series here but no information about the channels as I was expecting.

### `Session`

In [ ]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.session.mat"
file_path.is_file()


In [ ]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'session.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

This contains useful information like subject, animal and an epoch file.
Probably less relevant is information about other sources of data such as spikesorting and some of the analogous channels.

# Spikesorting 

## Testing CellExplorer

In [ ]:
from spikeinterface.extractors import CellExplorerSortingExtractor

file_path = session_files_path_list[0].parent / "e13_16f1_210302.spikes.cellinfo.mat"
try:
    extractor = CellExplorerSortingExtractor(file_path)
except Exception as e:
    print(e)

No `sessionInfo` for this file so the above throws an assertion. Let's see what each of the files.

The `CellExplorerSortingExtractor` uses the `sessionInfo` to extract the sampling frequency. But that might be somehwere else. 

Importantly, the files should contain the fields `UID` and `times` in a field called `spikes`. Let's see if any of the files contain this information and if it is consistent across them

After a second look and some comunication with Petersen it seems that we can use the `spikes.cellinfo` file. We can modify the `CellExplorerSortingExtractor` to use this file instead of the `sessionInfo` file to extract sampling rate.

## Individual spike files

### `spikes.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.spikes.cellinfo.mat"

mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'spikes.cellinfo.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

In [ ]:
spikes = mat_file["spikes"]
spikes.keys()

In [ ]:
sampling_rate = spikes["sr"]
cluster_id = spikes["cluID"]
times = spikes["times"]
unit_ids = spikes["UID"]

sampling_rate, cluster_id.shape, times.shape, unit_ids.shape, unit_ids[:3], cluster_id[:3], times[:3][0][:3]


### `cell_metrics.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.cell_metrics.cellinfo.mat"
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Ouput to an external file
with open(json_directory / 'cell_metrics.cellinfo.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

In [ ]:
times = mat_file["cell_metrics"]["spikes"]["times"]
unit_ids = mat_file["cell_metrics"]["UID"]
unit_ids = mat_file["cell_metrics"]["cluID"]
sampling_rate = None 

sampling_rate, cluster_id.shape, times.shape, unit_ids.shape, unit_ids[:3], cluster_id[:3], times[:3][0][:3]


### `mono_ress.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.mono_res.cellinfo.mat"
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'mono_res.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))
    


This is somet other analysis concerning `Monosynaptic connections.`

## Using cellexplorer with `spikes.cellinfo.mat`

In [ ]:
from spikeinterface.extractors import CellExplorerSortingExtractor

file_path = session_files_path_list[0].parent / "e13_16f1_210302.spikes.cellinfo.mat"

try:
    extractor = CellExplorerSortingExtractor(file_path, sampling_frequency=30_000.0)
except Exception as e:
    print(e)

In [ ]:
extractor.get_all_spike_trains()

## How to extract the sampling rate

In [ ]:
spikes_matfile_path = file_path
import scipy 
import hdf5storage

try:
    spikes_mat = scipy.io.loadmat(file_name=str(spikes_matfile_path))
    read_spikes_info_with_scipy = True
except NotImplementedError:
    spikes_mat = hdf5storage.loadmat(file_name=str(spikes_matfile_path))
    read_spikes_info_with_scipy = False
cell_info = spikes_mat.get("spikes", np.empty(0))
cell_info_fields = cell_info.dtype.names

sampling_rate = float(cell_info["sr"][0][0][0][0])